[View in Colaboratory](https://colab.research.google.com/github/YuexiSC/files/blob/master/HomeMortgage.ipynb)

## 1. Environment Configuration 

> <li>  Themis_ml package loading

In [2]:
!pip install themis_ml

In [0]:
from themis_ml import datasets
from themis_ml.datasets.german_credit_data_map import preprocess_german_credit_data
from themis_ml.metrics import mean_difference, normalized_mean_difference, \
    mean_confidence_interval

  > <li>  Google Token AUthorization





In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

> <li>  Setting with Auth for using Google GPU

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

 >  <li>  Package loading

In [0]:
import pandas as pd
import numpy as np
import io
import csv

##2. Cleaning Data

### 1. Subseting Data


>> Original data (HMDA14a, 53 columns *4172033 rows)
  
>> Choosing 28 columns

In [7]:
HMDA14a=pd.read_csv("drive/app/hmda14a.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,41,43,45,47,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
len(HMDA14a)

4172033

In [12]:
HMDA14a.head()

,YEAR,RESP_ID,AGENCY,LOAN_TYPE,PROP_TYPE,PURPOSE,OCCUPANCY,AMNT,PRE_APP,ACTION,...,MINPOP,MED_INCCH,MED_INC,TCT2MSACH,TCT2MSA,OWNOCCCH,OWNOCC,OWNOCC4CH,OWNOCC4,APP_FLAG
0,2014,0000001281,3,1,1,1,2,210,3,1,...,52.31,84900.0,84900,58.72,58.72,149.0,149,327.0,327,0
1,2014,0000001933,3,1,1,1,1,271,3,5,...,26.83,84900.0,84900,111.63,111.63,2926.0,2926,3554.0,3554,0
2,2014,0000002285,5,1,1,3,1,60,3,1,...,38.21,84900.0,84900,115.35,115.35,1778.0,1778,2199.0,2199,0
3,2014,0000002285,5,1,1,3,1,91,3,1,...,46.12,84900.0,84900,86.38,86.38,2152.0,2152,3002.0,3002,0
4,2014,0000002285,5,1,1,2,1,85,3,1,...,37.34,84900.0,84900,111.17,111.17,1892.0,1892,2177.0,2177,0


In [13]:
HMDA14a.columns

Index(['YEAR', 'RESP_ID', 'AGENCY', 'LOAN_TYPE', 'PROP_TYPE', 'PURPOSE',
       'OCCUPANCY', 'AMNT', 'PRE_APP', 'ACTION', 'MA', 'STATE', 'COUNTY',
       'TRACT', 'APP_ETH', 'COAPP_ETH', 'APP_RACE1', 'APP_RACE2', 'APP_RACE3',
       'APP_RACE4', 'APP_RACE5', 'COAP_RACE1', 'COAP_RACE2', 'COAP_RACE3',
       'COAP_RACE4', 'COAP_RACE5', 'APP_SEX', 'COAP_SEX', 'APP_INCCH',
       'APP_INC', 'PURCHTYPE', 'DENIAL1', 'DENIAL2', 'DENIAL3', 'SPREADCH',
       'SPREAD', 'HOEPA', 'LIENSTATUS', 'EDITSTATUS', 'SEQNO', 'POPCH', 'POP',
       'MINPOPCH', 'MINPOP', 'MED_INCCH', 'MED_INC', 'TCT2MSACH', 'TCT2MSA',
       'OWNOCCCH', 'OWNOCC', 'OWNOCC4CH', 'OWNOCC4', 'APP_FLAG'],
      dtype='object')

In [0]:
#feature selection
subset14a=HMDA14a.iloc[:,[0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,21,26,27,28,30,34,36,40,42,44,46,48,50]]

In [15]:
subset14a.head()

,YEAR,LOAN_TYPE,PROP_TYPE,PURPOSE,OCCUPANCY,AMNT,PRE_APP,ACTION,MA,STATE,...,APP_INCCH,PURCHTYPE,SPREADCH,HOEPA,POPCH,MINPOPCH,MED_INCCH,TCT2MSACH,OWNOCCCH,OWNOCC4CH
0,2014,1,1,1,2,210,3,1,11260.0,2,...,214.0,3,NaN,2,1988.0,52.31,84900.0,58.72,149.0,327.0
1,2014,1,1,1,1,271,3,5,11260.0,2,...,86.0,0,NaN,2,9670.0,26.83,84900.0,111.63,2926.0,3554.0
2,2014,1,1,3,1,60,3,1,11260.0,2,...,77.0,0,NaN,2,6179.0,38.21,84900.0,115.35,1778.0,2199.0
3,2014,1,1,3,1,91,3,1,11260.0,2,...,176.0,0,NaN,2,9068.0,46.12,84900.0,86.38,2152.0,3002.0
4,2014,1,1,2,1,85,3,1,11260.0,2,...,142.0,0,NaN,2,6341.0,37.34,84900.0,111.17,1892.0,2177.0


### 2. Data transformantion
> Actions; 

>Genders;

> Null Values;

#### a.  Target Variable: Action (Whether a loan can get approved)





*   1 = approved (Good)     
              2881765
*   0 = Not approved         
              715662



---

Action Taken: 
>1 -- Loan originated ( it mean the banks approved the loan and applicant get the money).

>2 -- Application approved but not accepted 

>3 -- Application denied by financial institution 

>4 -- Application withdrawn by applicant 

>5 -- File closed for incompleteness 

>6 -- Loan purchased by the institution 

>7 -- Preapproval request denied by financial institution 

>8 -- Preapproval request approved but not accepted (optional reporting) 

Transfer Target variable:

<li>Action 1,2,6 should treat it as approved, so let’s treat these  as Group A
<li>Action 3 should treat it as not approved, so treat them as Group B
<li>Action 4,5 ,7,8 should be filtered out, since we don’t know the result that bank approve it or not.



In [26]:
#Filter out the ACTION with 4/5/7/8 Values
subset14a=subset14a[(subset14a.ACTION!=4)&(subset14a.ACTION!=5)&(subset14a.ACTION!=7)&(subset14a.ACTION!=8)]
subset14a.ACTION.unique()

array([1, 6, 3, 2])

In [27]:
#Treat the Action 3 as not approved group, flag as 0. Otherwise treated as 1.
subset14= subset14a.copy()
subset14.loc[:,'ACTION'] = np.where(subset14.ACTION==3, 0,1)
subset14.ACTION.unique()

array([1, 0])

In [28]:
subset14.ACTION.value_counts()


1    2881765
0     715662
Name: ACTION, dtype: int64

#### c. Feature Gender Transformation




<li> Female : 0
<li> Male: 1
  

---

>**APP_SEX **
```
APP_SEX 
Male:   1    1078513
Female: 0     239016
```


>**COAP_SEX **
```
Male:   1    1070921
Female: 0     246608
```


---

*   APP_SEX
	>1 -- Male
  
	>2 -- Female
  
	>3 -- Information not provided by applicant in mail, Internet, or telephone application
  
	>4 -- Not applicable

*   COAP_SEX
> Gender of the co applicant. Same codes as APP_SEX, plus 5 -- No co-applicant 
	
Feature  Transformation:

<li>Female: APP_SEX==2;   COAP_SEX==2

<li>Male: APP_SEX==1;  COAP_SEX==1

<li> Filter: Others


In [0]:
#Filter out the APP_SEX AND COAP_SEX with not applicable/ missing Values
subset14_sex=subset14.copy()
subset14_sex=subset14_sex[(subset14_sex.APP_SEX==1)|(subset14_sex.APP_SEX==2)]
subset14_sex=subset14_sex[(subset14_sex.COAP_SEX==1)|(subset14_sex.COAP_SEX==2)]

In [0]:
subset14_sex.loc[:,'APP_SEX'] = np.where(subset14_sex.APP_SEX==2, 0,1)
subset14_sex.loc[:,'COAP_SEX'] = np.where(subset14_sex.COAP_SEX==2, 0,1)

In [32]:
subset14_sex.APP_SEX.value_counts()

1    1078513
0     239016
Name: APP_SEX, dtype: int64

In [33]:
subset14_sex.COAP_SEX.value_counts()

0    1070921
1     246608
Name: COAP_SEX, dtype: int64

#### c.Null values

In [0]:
#Function to get null number and null percentage for each feature(who has the null values)
def countnull(countdf):
    colname_list=[]
    True_list=[]
    False_list=[]
    for idx,i in zip(countdf.columns, range(0,len(countdf.columns))):
        if countdf.iloc[:,i].isnull().value_counts()[0] != len(countdf):
            colname_list.append(idx)
            True_list.append(countdf.iloc[:,i].isnull().value_counts()[1])
            False_list.append(countdf.iloc[:,i].isnull().value_counts()[0])
    count_null=pd.DataFrame({'colname_list': colname_list,'Null': True_list,'Not_Null': False_list})
    count_null['Null_Percentage']= count_null.Null*1.0/(count_null.Not_Null+count_null.Null)
    Count_table= count_null.sort_values(by='Null_Percentage', ascending=False)
    return Count_table

In [36]:
countnull(subset14_sex)

,Not_Null,Null,colname_list,Null_Percentage
4,61419,1256110,SPREADCH,0.953383
0,1219958,97571,MA,0.074056
3,1259041,58488,APP_INCCH,0.044392
10,1311081,6448,OWNOCC4CH,0.004894
9,1311082,6447,OWNOCCCH,0.004893
8,1311104,6425,TCT2MSACH,0.004877
5,1311235,6294,POPCH,0.004777
6,1311235,6294,MINPOPCH,0.004777
2,1311398,6131,TRACT,0.004653
7,1311398,6131,MED_INCCH,0.004653


## 3. The Themis-ML library 

In [0]:
hmda=subset14_sex.copy()

###Set Targeted variable as:  
>*ACTION*  :  1/0   **and  measure Bias in **


>1.   Gender    
>2.   Ethinicity
>3.   Race



In [43]:
loan_approval=hmda.ACTION
loan_approval.value_counts()

1    1071320
0     246209
Name: ACTION, dtype: int64

#### 1. Protected Class: Sex 
  

*  advantaged group: Male
*  disadvantaged group: Female



In [45]:
app_gender=hmda.APP_SEX
app_gender.value_counts()

1    1078513
0     239016
Name: APP_SEX, dtype: int64

In [46]:
def report_metric(metric, mean_diff, lower, upper):
    print("{metric}: {md:0.02f} - 95% CI [{lower:0.02f}, {upper:0.02f}]"
          .format(metric=metric, md=mean_diff, lower=lower, upper=upper))

report_metric(
    "mean difference",
    *map(lambda x: x * 100, mean_difference(loan_approval, app_gender)))
report_metric(
    "normalized mean difference",
    *map(lambda x: x * 100, normalized_mean_difference(loan_approval, app_gender)))

mean difference: -5.45 - 95% CI [-5.63, -5.28]
normalized mean difference: -23.89 - 95% CI [-24.06, -23.72]


#### 2. Measure Ethinicity Bias



#### 3. Measure Race Bias

1.   Hispanic 
2.   Black 
3.   Asian 
4.   White

## Other: Cleaned dataset (Julia version)

In [33]:
HMDA_cleaned=pd.read_csv("drive/app/Clean_data.csv")

Unnamed: 0        0.00
YEAR           2014.00
LOAN_TYPE         1.00
PROP_TYPE         1.00
PURPOSE           1.00
OCCUPANCY         2.00
AMNT            210.00
PRE_APP           3.00
ACTION            1.00
MA                1.00
STATE             2.00
COUNTY           20.00
TRACT             5.00
APP_ETH           2.00
COAPP_ETH         5.00
APP_RACE1         5.00
COAP_RACE1        8.00
APP_SEX           1.00
COAP_SEX          5.00
APP_INCCH       214.00
PURCHTYPE         3.00
HOEPA             2.00
POPCH          1988.00
MINPOPCH         52.31
MED_INCCH     84900.00
TCT2MSACH        58.72
OWNOCCCH        149.00
OWNOCC4CH       327.00
action            1.00
female            0.00
hispanic          0.00
black             0.00
co_black          0.00
Name: 0, dtype: float64

In [32]:
HMDA_cleaned.columns

Index(['Unnamed: 0', 'YEAR', 'LOAN_TYPE', 'PROP_TYPE', 'PURPOSE', 'OCCUPANCY',
       'AMNT', 'PRE_APP', 'ACTION', 'MA', 'STATE', 'COUNTY', 'TRACT',
       'APP_ETH', 'COAPP_ETH', 'APP_RACE1', 'COAP_RACE1', 'APP_SEX',
       'COAP_SEX', 'APP_INCCH', 'PURCHTYPE', 'HOEPA', 'POPCH', 'MINPOPCH',
       'MED_INCCH', 'TCT2MSACH', 'OWNOCCCH', 'OWNOCC4CH', 'action', 'female',
       'hispanic', 'black', 'co_black'],
      dtype='object')

In [17]:
HMDA_cleaned

,Unnamed: 0,YEAR,LOAN_TYPE,PROP_TYPE,PURPOSE,OCCUPANCY,AMNT,PRE_APP,ACTION,MA,...,MINPOPCH,MED_INCCH,TCT2MSACH,OWNOCCCH,OWNOCC4CH,action,female,hispanic,black,co_black
0,0,2014,1,1,1,2,210,3,1,1,...,52.31,84900.0,58.72,149.0,327.0,1,0,0,0,0
1,3,2014,1,1,3,1,91,3,1,1,...,46.12,84900.0,86.38,2152.0,3002.0,1,0,0,0,0
2,4,2014,1,1,2,1,85,3,1,1,...,37.34,84900.0,111.17,1892.0,2177.0,1,0,0,0,0
3,5,2014,1,1,1,1,301,1,1,1,...,44.27,84900.0,87.68,1734.0,1829.0,1,1,0,0,0
4,7,2014,1,1,3,1,245,3,1,1,...,17.51,84900.0,129.43,3080.0,3541.0,1,0,0,0,0
5,8,2014,1,1,1,1,275,1,1,1,...,39.39,84900.0,95.77,751.0,1172.0,1,0,0,0,0
6,9,2014,1,1,3,2,233,3,1,1,...,19.20,84900.0,99.63,1446.0,1781.0,1,1,0,0,0
7,10,2014,1,1,1,1,295,1,1,1,...,16.63,84900.0,111.23,1593.0,1754.0,1,0,0,0,0
8,12,2014,1,1,1,1,288,1,1,1,...,27.58,84900.0,123.83,2049.0,2230.0,1,0,0,0,0
9,13,2014,1,1,3,1,278,3,1,1,...,26.83,84900.0,111.63,2926.0,3554.0,1,1,0,0,0
